# Metrics calculation

In [1]:
import pandas as pd
# summary_dataset = pd.read_csv("summary_dataset_with_bertscore.csv")
# columns = [column for column in summary_dataset.columns if "bertscore" not in column] 
# print(columns)
# summary_dataset = summary_dataset[columns]
# summary_dataset.to_csv("summary_dataset.csv", index=False)
summary_dataset = pd.read_csv("summary_dataset.csv")
summary_dataset.head()

FileNotFoundError: [Errno 2] No such file or directory: 'summary_dataset.csv'

## BERTScore

In [ ]:
from evaluate import load
bertscore = load("bertscore")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt"
]

In [ ]:
results = {}
references = list(summary_dataset["summary"])
for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    results[predictions_field] = bertscore.compute(predictions=predictions, references=references, lang="ru", verbose=True, model_type="microsoft/mdeberta-v3-base", nthreads=16, batch_size=16, use_fast_tokenizer=False)

In [ ]:
data_keys = list(results.keys())
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_bertscore_" + old_metric_key] = results[data_key].pop(old_metric_key)

In [ ]:
print(results[filename.split(".")[0]].keys())

In [ ]:
print(len(results[filename.split(".")[0]][f"{filename.split('.')[0]}_bertscore_precision"]))

In [ ]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

In [ ]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df], axis=1)
new_summary_dataset.head()

In [ ]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [ ]:
summary_dataset.head()

In [ ]:
from statistics import mean 
from collections import defaultdict
score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    
    mean_f1 = mean(results[predictions_field][f"{predictions_field}_bertscore_f1"])
    var_f1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_f1"])) - mean_f1 ** 2
    
    mean_precision = mean(results[predictions_field][f"{predictions_field}_bertscore_precision"])
    var_precision = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_precision"])) - mean_precision ** 2
    
    mean_recall = mean(results[predictions_field][f"{predictions_field}_bertscore_recall"])
    var_recall = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_recall"])) - mean_recall ** 2
    
    score_stats[predictions_field].update({"mean_precision": mean_precision, "var_precision": var_precision, "mean_recall": mean_recall, "var_recall": var_recall, "mean_f1": mean_f1, "var_f1": var_f1})
print(dict(score_stats))

In [ ]:
results.keys()

In [ ]:
stats_pd = pd.DataFrame(score_stats)

In [ ]:
stats_pdsummary_dataset_for_expert_evaluation.head(None)

In [ ]:
summary_dataset.to_csv("summary_dataset_with_bertscore.csv", index=False)

## BLEU

In [ ]:
import razdel
from evaluate import load
# results = {}
bleu = load('bleu')
references = list(summary_dataset["summary"])

def a(x):
    tokens = list(razdel.tokenize(x))
    return [_.text for _ in tokens]
    # return x.split()

for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    # results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=lambda x : list(razdel.tokenize(x)))
    results[predictions_field]["bleu"] = []
    for ref, pred in tqdm(zip(references, predictions)):
        bl = bleu.compute(predictions=[pred], references=[ref], tokenizer=a)
        results[predictions_field]["bleu"].append(bl["bleu"])

In [213]:
data_keys = list(results.keys())
print(data_keys)
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    print(old_keys)
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_" + old_metric_key] = results[data_key].pop(old_metric_key)

['mbart_predictions', 'mt5_predictions', 'summarunner_predictions']
['bleu', 'precisions', 'brevity_penalty', 'length_ratio', 'translation_length', 'reference_length']
['bleu', 'precisions', 'brevity_penalty', 'length_ratio', 'translation_length', 'reference_length']
['bleu', 'precisions', 'brevity_penalty', 'length_ratio', 'translation_length', 'reference_length']


In [214]:
print(results[filename.split(".")[0]].keys())

dict_keys(['summarunner_predictions_bleu', 'summarunner_predictions_precisions', 'summarunner_predictions_brevity_penalty', 'summarunner_predictions_length_ratio', 'summarunner_predictions_translation_length', 'summarunner_predictions_reference_length'])


In [215]:
print(results[filename.split(".")[0]][f"{filename.split('.')[0]}_bleu"])

0.01688847827402911


In [216]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if not "bleu" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu
0,0.017181,0.005554,0.016888


In [217]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df], axis=1)
new_summary_dataset.head()

,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,mt5_predictions_bertscore_recall,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu
0,wikilingua_multilingual-val-36832,"Есть широкий выбор сортов картофеля, обычно их...",Запланируйте посадку картофеля в соответствии ...,Российский картофель — один из самых популярны...,Созревание картофеля может принесут урожай в т...,с помощью лопаты аккуратно добавьте в горшок р...,0.575211,0.588802,0.581927,0.593385,0.577867,0.585523,0.708673,0.670665,0.689146,0.017181,0.005554,0.016888
1,wikilingua_multilingual-val-36833,"Вы не сможете найти красного люциана без кожи,...",Используйте филе без кожи. Подготовьте жидкое ...,"Начните приготовление красного люциана, которы...",Японская панировка красного люциана - самая по...,попробуйте положить филе и тесто вместе в паке...,0.646338,0.753633,0.695874,0.603925,0.585239,0.594435,0.647352,0.715943,0.679922,NaN,NaN,NaN
2,wikilingua_multilingual-val-36834,Груши должны быть красно-оранжевого или пурпур...,Купите или вырастите грушу. Удалите шипы. Очис...,"В сегодняшнем обзоре мы расскажем о том, как с...",Груши с белой кожурой - одна из самых популярн...,режьте вдоль верхней и нижней части груши по ц...,0.589720,0.599384,0.594513,0.562461,0.570675,0.566538,0.637965,0.709592,0.671875,NaN,NaN,NaN
3,wikilingua_multilingual-val-36835,Со светлыми волосами вам придется изменить сво...,"Учтите образ жизни. Подумайте о том, насколько...",Со светлыми волосами вам придется изменить сво...,"Если вы хотите стать блондинкой, то наверняка ...","скажите им, что вы хотите стать блондинкой, и ...",0.607002,0.655611,0.630370,0.646967,0.590495,0.617442,0.606263,0.660061,0.632019,NaN,NaN,NaN
4,wikilingua_multilingual-val-36836,Если мяч уходит немного влево (у игроков с пра...,Откорректируйте слайс. Откорректируйте хук. Ис...,В преддверии матча чемпионата мира по хоккею с...,"Если мяч летит немного вправо, а затем резко н...",обратите внимание на свой хват. чтобы убедитьс...,0.528840,0.520251,0.524510,0.592174,0.616999,0.604332,0.574747,0.636400,0.604004,NaN,NaN,NaN


In [218]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [219]:
summary_dataset.head()

,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,mt5_predictions_bertscore_recall,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu
0,wikilingua_multilingual-val-36832,"Есть широкий выбор сортов картофеля, обычно их...",Запланируйте посадку картофеля в соответствии ...,Российский картофель — один из самых популярны...,Созревание картофеля может принесут урожай в т...,с помощью лопаты аккуратно добавьте в горшок р...,0.575211,0.588802,0.581927,0.593385,0.577867,0.585523,0.708673,0.670665,0.689146,0.017181,0.005554,0.016888
1,wikilingua_multilingual-val-36833,"Вы не сможете найти красного люциана без кожи,...",Используйте филе без кожи. Подготовьте жидкое ...,"Начните приготовление красного люциана, которы...",Японская панировка красного люциана - самая по...,попробуйте положить филе и тесто вместе в паке...,0.646338,0.753633,0.695874,0.603925,0.585239,0.594435,0.647352,0.715943,0.679922,NaN,NaN,NaN
2,wikilingua_multilingual-val-36834,Груши должны быть красно-оранжевого или пурпур...,Купите или вырастите грушу. Удалите шипы. Очис...,"В сегодняшнем обзоре мы расскажем о том, как с...",Груши с белой кожурой - одна из самых популярн...,режьте вдоль верхней и нижней части груши по ц...,0.589720,0.599384,0.594513,0.562461,0.570675,0.566538,0.637965,0.709592,0.671875,NaN,NaN,NaN
3,wikilingua_multilingual-val-36835,Со светлыми волосами вам придется изменить сво...,"Учтите образ жизни. Подумайте о том, насколько...",Со светлыми волосами вам придется изменить сво...,"Если вы хотите стать блондинкой, то наверняка ...","скажите им, что вы хотите стать блондинкой, и ...",0.607002,0.655611,0.630370,0.646967,0.590495,0.617442,0.606263,0.660061,0.632019,NaN,NaN,NaN
4,wikilingua_multilingual-val-36836,Если мяч уходит немного влево (у игроков с пра...,Откорректируйте слайс. Откорректируйте хук. Ис...,В преддверии матча чемпионата мира по хоккею с...,"Если мяч летит немного вправо, а затем резко н...",обратите внимание на свой хват. чтобы убедитьс...,0.528840,0.520251,0.524510,0.592174,0.616999,0.604332,0.574747,0.636400,0.604004,NaN,NaN,NaN


In [220]:
# from statistics import mean 
# from collections import defaultdict
# score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
# for filename in files:
#     predictions_field = filename.split(".")[0]
#     mean_rouge1 = mean(results[predictions_field][f"{predictions_field}_rouge1"])
#     var_rouge1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge1"])) - mean_rouge1 ** 2
    
#     mean_rouge2 = mean(results[predictions_field][f"{predictions_field}_rouge2"])
#     var_rouge2 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge2"])) - mean_rouge2 ** 2
    
#     mean_rougeL = mean(results[predictions_field][f"{predictions_field}_rougeL"])
#     var_rougeL = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeL"])) - mean_rougeL ** 2
    
#     mean_rougeLsum = mean(results[predictions_field][f"{predictions_field}_rougeLsum"])
#     var_rougeLsum = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeLsum"])) - mean_rougeLsum ** 2
#     score_stats[predictions_field].update({"mean_rouge1": mean_rouge1, "var_rouge1": var_rouge1, "mean_rouge2": mean_rouge2, "var_rouge2": var_rouge2, "mean_rougeL": mean_rougeL, "var_rougeL": var_rougeL, "mean_rougeLsum": mean_rougeLsum, "var_rougeLsum": var_rougeLsum})
# print(dict(score_stats))

In [221]:
# stats_pd = pd.DataFrame(score_stats)

In [222]:
# stats_pd.head(len(stats_pd))

In [223]:
summary_dataset.to_csv("summary_dataset_with_bertscore_bleu.csv", index=False)

## ROUGE

In [4]:
import pandas as pd
summary_dataset = pd.read_csv("summary_dataset_with_bertscore_bleu.csv")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt"
]

In [22]:
import razdel
from evaluate import load
# results = {}
rouge = load('rouge')
references = list(summary_dataset["summary"])

def a(x):
    tokens = list(razdel.tokenize(x))
    return [_.text for _ in tokens]
    # return x.split()

for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    # results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=lambda x : list(razdel.tokenize(x)))
    results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=a)

In [23]:
data_keys = list(results.keys())
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_" + old_metric_key] = results[data_key].pop(old_metric_key)

In [24]:
print(results[filename.split(".")[0]].keys())

dict_keys(['summarunner_predictions_rouge1', 'summarunner_predictions_rouge2', 'summarunner_predictions_rougeL', 'summarunner_predictions_rougeLsum'])


In [25]:
print(len(results[filename.split(".")[0]][f"{filename.split('.')[0]}_rouge1"]))

6793


In [26]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_rouge1,mbart_predictions_rouge2,mbart_predictions_rougeL,mbart_predictions_rougeLsum,mt5_predictions_rouge1,mt5_predictions_rouge2,mt5_predictions_rougeL,mt5_predictions_rougeLsum,summarunner_predictions_rouge1,summarunner_predictions_rouge2,summarunner_predictions_rougeL,summarunner_predictions_rougeLsum
0,0.292683,0.099174,0.178862,0.178862,0.317073,0.050000,0.146341,0.146341,0.272727,0.092593,0.181818,0.181818
1,0.258621,0.052632,0.189655,0.189655,0.164706,0.048193,0.164706,0.164706,0.222222,0.034783,0.153846,0.153846
2,0.358974,0.191304,0.324786,0.324786,0.135593,0.035088,0.135593,0.135593,0.306306,0.110092,0.252252,0.252252
3,0.377953,0.176000,0.299213,0.299213,0.264706,0.090909,0.176471,0.176471,0.346457,0.128000,0.267717,0.267717
4,0.436975,0.290598,0.352941,0.352941,0.602740,0.422535,0.465753,0.465753,0.391304,0.220588,0.289855,0.289855


In [28]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df], axis=1)
new_summary_dataset.head()

,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,...,mbart_predictions_rougeL,mbart_predictions_rougeLsum,mt5_predictions_rouge1,mt5_predictions_rouge2,mt5_predictions_rougeL,mt5_predictions_rougeLsum,summarunner_predictions_rouge1,summarunner_predictions_rouge2,summarunner_predictions_rougeL,summarunner_predictions_rougeLsum
0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,0.758551,...,0.178862,0.178862,0.317073,0.050000,0.146341,0.146341,0.272727,0.092593,0.181818,0.181818
1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,0.744267,...,0.189655,0.189655,0.164706,0.048193,0.164706,0.164706,0.222222,0.034783,0.153846,0.153846
2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707951,0.761381,...,0.324786,0.324786,0.135593,0.035088,0.135593,0.135593,0.306306,0.110092,0.252252,0.252252
3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,0.772861,...,0.299213,0.299213,0.264706,0.090909,0.176471,0.176471,0.346457,0.128000,0.267717,0.267717
4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,0.896744,...,0.352941,0.352941,0.602740,0.422535,0.465753,0.465753,0.391304,0.220588,0.289855,0.289855


In [29]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [30]:
summary_dataset.head()

,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,...,mbart_predictions_rougeL,mbart_predictions_rougeLsum,mt5_predictions_rouge1,mt5_predictions_rouge2,mt5_predictions_rougeL,mt5_predictions_rougeLsum,summarunner_predictions_rouge1,summarunner_predictions_rouge2,summarunner_predictions_rougeL,summarunner_predictions_rougeLsum
0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,0.758551,...,0.178862,0.178862,0.317073,0.050000,0.146341,0.146341,0.272727,0.092593,0.181818,0.181818
1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,0.744267,...,0.189655,0.189655,0.164706,0.048193,0.164706,0.164706,0.222222,0.034783,0.153846,0.153846
2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707951,0.761381,...,0.324786,0.324786,0.135593,0.035088,0.135593,0.135593,0.306306,0.110092,0.252252,0.252252
3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,0.772861,...,0.299213,0.299213,0.264706,0.090909,0.176471,0.176471,0.346457,0.128000,0.267717,0.267717
4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,0.896744,...,0.352941,0.352941,0.602740,0.422535,0.465753,0.465753,0.391304,0.220588,0.289855,0.289855


Посчитаем статистики для представления о качестве датасета

In [31]:
from statistics import mean 
from collections import defaultdict
score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    mean_rouge1 = mean(results[predictions_field][f"{predictions_field}_rouge1"])
    var_rouge1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge1"])) - mean_rouge1 ** 2
    
    mean_rouge2 = mean(results[predictions_field][f"{predictions_field}_rouge2"])
    var_rouge2 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge2"])) - mean_rouge2 ** 2
    
    mean_rougeL = mean(results[predictions_field][f"{predictions_field}_rougeL"])
    var_rougeL = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeL"])) - mean_rougeL ** 2
    
    mean_rougeLsum = mean(results[predictions_field][f"{predictions_field}_rougeLsum"])
    var_rougeLsum = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeLsum"])) - mean_rougeLsum ** 2
    score_stats[predictions_field].update({"mean_rouge1": mean_rouge1, "var_rouge1": var_rouge1, "mean_rouge2": mean_rouge2, "var_rouge2": var_rouge2, "mean_rougeL": mean_rougeL, "var_rougeL": var_rougeL, "mean_rougeLsum": mean_rougeLsum, "var_rougeLsum": var_rougeLsum})
print(dict(score_stats))

{'mbart_predictions': {'mean_precision': 0.69237233716575, 'var_precision': 0.002488098740164313, 'mean_recall': 0.6962039852359939, 'var_recall': 0.0027879992057889824, 'mean_f1': 0.6936039480372626, 'var_f1': 0.0021470763656115666, 'mean_rouge1': 0.31801428929276543, 'var_rouge1': 0.009495205444211535, 'mean_rouge2': 0.11031830313712905, 'var_rouge2': 0.008153947814415904, 'mean_rougeL': 0.22786116051309793, 'var_rougeL': 0.007198383393708874, 'mean_rougeLsum': 0.22786116051309793, 'var_rougeLsum': 0.007198383393708874}, 'mt5_predictions': {'mean_precision': 0.7240416967129999, 'var_precision': 0.004430899438510272, 'mean_recall': 0.6168432765166363, 'var_recall': 0.002580475194013665, 'mean_f1': 0.6652786503672757, 'var_f1': 0.0027420287641276575, 'mean_rouge1': 0.21508826169541578, 'var_rouge1': 0.009881811467355672, 'mean_rouge2': 0.07156625942695444, 'var_rouge2': 0.005848079935605663, 'mean_rougeL': 0.17260913142505446, 'var_rougeL': 0.006988653580895881, 'mean_rougeLsum': 0.172

In [32]:
stats_pd = pd.DataFrame(score_stats)

In [33]:
stats_pd.head(len(stats_pd))

,mbart_predictions,mt5_predictions,summarunner_predictions
mean_precision,0.692372,0.724042,0.659372
var_precision,0.002488,0.004431,0.001887
mean_recall,0.696204,0.616843,0.671184
var_recall,0.002788,0.002580,0.002593
mean_f1,0.693604,0.665279,0.664570
var_f1,0.002147,0.002742,0.001770
mean_rouge1,0.318014,0.215088,0.266036
var_rouge1,0.009495,0.009882,0.006447
mean_rouge2,0.110318,0.071566,0.065286
var_rouge2,0.008154,0.005848,0.004327


In [34]:
summary_dataset.to_csv("summary_dataset_with_bertscore_rouge.csv", index=False)

## METEOR

https://huggingface.co/spaces/evaluate-metric/meteor

In [35]:
import pandas as pd
summary_dataset = pd.read_csv("summary_dataset_with_bertscore_bleu_rouge.csv")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
]

In [36]:
from tqdm import tqdm
from threading import Thread
from evaluate import load
import os, contextlib


def calculate_meteor(result_list, predictions, references, thread_no, step):
    meteor = None
    # with open(os.devnull, 'w') as devnull:
    #     with contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
    meteor = load('meteor')
    index = thread_no
    disable_tqdm = (thread_no != 0)
    
    with tqdm(total=len(predictions), disable=disable_tqdm) as pbar:
        pbar.update(index)
        while index < len(predictions):
            result_list[index] = meteor.compute(predictions=[predictions[index]], references=[references[index]])
            index += step
            pbar.update(step)
    print(f"Thread {thread_no} finished")

results = {}
references = list(summary_dataset["summary"])
for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    results[predictions_field] = [0] * len(predictions)
    
    thread_num = 32
    thread_pool = []
    for thread in range(thread_num):
        thread_pool.append(Thread(target=calculate_meteor, args=[results[predictions_field], predictions, references, thread, thread_num], daemon=True))
        thread_pool[-1].start()

    for thread in range(thread_num):
        thread_pool[thread].join()

[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_da

Thread 11 finished


 79%|█████████████████████████████▎       | 5376/6793 [00:08<00:01, 1020.82it/s]

Thread 23 finished


 89%|█████████████████████████████████▋    | 6016/6793 [00:09<00:01, 635.30it/s]

...

Thread 21 finished
Thread 30 finished
Thread 13 finished


 85%|████████████████████████████████▍     | 5792/6793 [00:11<00:01, 578.09it/s]

Thread 12 finished
Thread 14 finished
Thread 24 finished
Thread 26 finished
Thread 1 finished


6816it [00:12, 566.86it/s]                                                      

Thread 8 finished
Thread 2 finished
Thread 5 finished
Thread 15 finished
Thread 28 finished
Thread 0 finished
Thread 27 finished
Thread 18 finished


In [37]:
for key in results.keys():
    results[key] = {f"{key}_meteor": [val["meteor"] for val in results[key]]}
    print(len(results[key]))

1
1
1


In [38]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor
0,0.327036,0.209577,0.298614
1,0.155565,0.087664,0.140027
2,0.332507,0.069983,0.336473
3,0.399261,0.186599,0.404262
4,0.601744,0.576856,0.570060


In [39]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df], axis=1)
new_summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,...,mt5_predictions_rouge2,mt5_predictions_rougeL,mt5_predictions_rougeLsum,summarunner_predictions_rouge1,summarunner_predictions_rouge2,summarunner_predictions_rougeL,summarunner_predictions_rougeLsum,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,...,0.050000,0.146341,0.146341,0.272727,0.092593,0.181818,0.181818,0.327036,0.209577,0.298614
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,...,0.048193,0.164706,0.164706,0.222222,0.034783,0.153846,0.153846,0.155565,0.087664,0.140027
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707951,...,0.035088,0.135593,0.135593,0.306306,0.110092,0.252252,0.252252,0.332507,0.069983,0.336473
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,...,0.090909,0.176471,0.176471,0.346457,0.128000,0.267717,0.267717,0.399261,0.186599,0.404262
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,...,0.422535,0.465753,0.465753,0.391304,0.220588,0.289855,0.289855,0.601744,0.576856,0.570060


In [40]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [41]:
summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,...,mt5_predictions_rouge2,mt5_predictions_rougeL,mt5_predictions_rougeLsum,summarunner_predictions_rouge1,summarunner_predictions_rouge2,summarunner_predictions_rougeL,summarunner_predictions_rougeLsum,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,...,0.050000,0.146341,0.146341,0.272727,0.092593,0.181818,0.181818,0.327036,0.209577,0.298614
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,...,0.048193,0.164706,0.164706,0.222222,0.034783,0.153846,0.153846,0.155565,0.087664,0.140027
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707951,...,0.035088,0.135593,0.135593,0.306306,0.110092,0.252252,0.252252,0.332507,0.069983,0.336473
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,...,0.090909,0.176471,0.176471,0.346457,0.128000,0.267717,0.267717,0.399261,0.186599,0.404262
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,...,0.422535,0.465753,0.465753,0.391304,0.220588,0.289855,0.289855,0.601744,0.576856,0.570060


Посчитаем статистики для представления о качестве датасета

In [42]:
from collections import defaultdict

score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    score_stats[predictions_field].update({f"meteor": sum(results[predictions_field][f"{predictions_field}_meteor"]) / len(results[predictions_field][f"{predictions_field}_meteor"])})

In [43]:
stats_pd = pd.DataFrame(score_stats)

In [44]:
stats_pd.head(len(stats_pd))

,mbart_predictions,mt5_predictions,summarunner_predictions
mean_precision,0.692372,0.724042,0.659372
var_precision,0.002488,0.004431,0.001887
mean_recall,0.696204,0.616843,0.671184
var_recall,0.002788,0.002580,0.002593
mean_f1,0.693604,0.665279,0.664570
var_f1,0.002147,0.002742,0.001770
mean_rouge1,0.318014,0.215088,0.266036
var_rouge1,0.009495,0.009882,0.006447
mean_rouge2,0.110318,0.071566,0.065286
var_rouge2,0.008154,0.005848,0.004327


In [45]:
summary_dataset.to_csv("summary_dataset_with_bertscore_bleu_rouge_meteor.csv", index=False)